In [ ]:
from bs4 import BeautifulSoup as bs
import lxml
import os

In [ ]:
path = "dcp-data/letters/"
files = os.listdir(path)
files = files[1:]
print(files[1])
for file_target in files:
    with open(path+file_target,"r",encoding="utf8") as file:
        print(file)
        print(file_target)
        content = file.readlines()
    #     print(content[0])
        content = "".join(content)
        bs_content = bs(content, "lxml")
#         print(bs_content.find_all("div")[0])
        print(bs_content.find_all("div")[0].find_all("div")[1].p)
        print("")
        bs_2 = bs_content.find_all("div")[0].find_all("div")[1].p.text
        print(bs_2)
        break
test = bs_content.find_all("title")[0]
print(test)